In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_final = pd.read_excel("data/processed/df_final.xlsx")

print(df_final.columns)

Index(['Unnamed: 0', 'Date', 'Ticker', 'dolvol_lag2', 'maxret', 'retvol',
       'mom36m', 'mom12m', 'mom6m', 'mom1m', 'chmom', 'turn', 'indmom',
       'baspread', 'illiq', 'stdturn', 'beta', 'beta_squared', 'idiovol',
       'mvel1', 'agr', 'cashpr', 'chinv', 'chsh', 'depr', 'dy', 'ep', 'invest',
       'rd_mve', 'sp', 'nincr', 'excess_return'],
      dtype='object')


In [ ]:
df_final = df_final.sort_values("Date").reset_index(drop=True)  # reset pour que les indices soient bien continus

# Paramètres

#split en 85 / 15% 
dates = sorted(df_final["Date"].unique())
total_months = len(dates)               # 360 mois 
train_initial = int(total_months * 0.85)  # 85% = ~306 mois
step = 12                               # 12 mois = 1 an

splits = []

#cette boucle for va créer les fenêtres glissantes
for i in range(0, total_months - train_initial - 2 * step + 1, step): #start / stop / step 
    
    #découpage du temps pour les fenêtres 
    train_end = train_initial + i
    val_start = train_end
    val_end = val_start + step
    test_start = val_end
    test_end = test_start + step

    if test_end > total_months:
        break
    
    #création index : je prends dans la liste "dates" les dates d'entraînement, de validation et de test en fct de leur position dans la liste
    #permet de couper selon les dates et non les indices
    train_dates = dates[:train_end] 
    val_dates = dates[val_start:val_end]
    test_dates = dates[test_start:test_end]

    #ici je cherche les dates dans df qui correspondent aux dates de train, val et test
    train_idx = df_final[df_final["Date"].isin(train_dates)].index.tolist()
    val_idx = df_final[df_final["Date"].isin(val_dates)].index.tolist()
    test_idx = df_final[df_final["Date"].isin(test_dates)].index.tolist()

    #on ajoute chaque triplet dans la liste splits 
    splits.append((train_idx, val_idx, test_idx))


Train : 1990-12 → 2016-05 | Val : 2016-06 → 2017-05 | Test : 2017-06 → 2018-05
Train : 1990-12 → 2017-05 | Val : 2017-06 → 2018-05 | Test : 2018-06 → 2019-05
Train : 1990-12 → 2018-05 | Val : 2018-06 → 2019-05 | Test : 2019-06 → 2020-05


In [15]:
print(splits.head())

AttributeError: 'list' object has no attribute 'head'

In [10]:
# Initialisation des listes pour stocker résultats
mse_list = []
r2_list = []


# x : variables explicatives 
exclude_cols = ["Ticker", "Date", "excess_return"]

for train_idx, val_idx, test_idx in splits:
    # Construction des jeux
    df_train = df_final.loc[train_idx]
    df_val = df_final.loc[val_idx]
    df_test = df_final.loc[test_idx]

    #X et Y
    X_train = df_train.drop(columns=exclude_cols)
    Y_train = df_train["excess_return"]

    X_val = df_val.drop(columns=exclude_cols)
    y_val = df_val["excess_return"]
    
    X_test = df_test.drop(columns=exclude_cols)
    y_test = df_test["excess_return"]

    # Entraînement sur train + validation (pour maximiser données d’apprentissage)
    X_train_val = pd.concat([X_train, X_val])
    y_train_val = pd.concat([y_train, y_val])
    

Index(['Unnamed: 0', 'dolvol_lag2', 'maxret', 'retvol', 'mom36m', 'mom12m',
       'mom6m', 'mom1m', 'chmom', 'turn', 'indmom', 'baspread', 'illiq',
       'stdturn', 'beta', 'beta_squared', 'idiovol', 'mvel1', 'agr', 'cashpr',
       'chinv', 'chsh', 'depr', 'dy', 'ep', 'invest', 'rd_mve', 'sp', 'nincr'],
      dtype='object')
Index(['Unnamed: 0', 'dolvol_lag2', 'maxret', 'retvol', 'mom36m', 'mom12m',
       'mom6m', 'mom1m', 'chmom', 'turn', 'indmom', 'baspread', 'illiq',
       'stdturn', 'beta', 'beta_squared', 'idiovol', 'mvel1', 'agr', 'cashpr',
       'chinv', 'chsh', 'depr', 'dy', 'ep', 'invest', 'rd_mve', 'sp', 'nincr'],
      dtype='object')
True


In [ ]:



    
    # Entraînement sur train + validation (pour maximiser données d’apprentissage)
    X_train_val = pd.concat([X_train, X_val])
    y_train_val = pd.concat([y_train, y_val])
    
    model = LinearRegression()
    model.fit(X_train_val, y_train_val)
    
    # Prédiction sur test
    y_pred = model.predict(X_test)
    
    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    r2 = model.score(X_test, y_test)
    
    mse_list.append(mse)
    r2_list.append(r2)

# Résultats globaux
print(f"Mean MSE over all splits: {np.mean(mse_list):.5f}")
print(f"Mean R2 over all splits: {np.mean(r2_list):.5f}")



ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values